# Entity Extraction+Linking using modern SciSpaCy models

## Initialize Dask Clusters

In [1]:
from dask_saturn.core import describe_sizes

describe_sizes()

{'medium': 'Medium - 2 cores - 4 GB RAM',
 'large': 'Large - 2 cores - 16 GB RAM',
 'xlarge': 'XLarge - 4 cores - 32 GB RAM',
 '2xlarge': '2XLarge - 8 cores - 64 GB RAM',
 '4xlarge': '4XLarge - 16 cores - 128 GB RAM',
 '8xlarge': '8XLarge - 32 cores - 256 GB RAM',
 '12xlarge': '12XLarge - 48 cores - 384 GB RAM',
 '16xlarge': '16XLarge - 64 cores - 512 GB RAM',
 'g4dnxlarge': 'T4-XLarge - 4 cores - 16 GB RAM - 1 GPU',
 'g4dn4xlarge': 'T4-4XLarge - 16 cores - 64 GB RAM - 1 GPU',
 'g4dn8xlarge': 'T4-8XLarge - 32 cores - 128 GB RAM - 1 GPU',
 'p32xlarge': 'V100-2XLarge - 8 cores - 61 GB RAM - 1 GPU',
 'p38xlarge': 'V100-8XLarge - 32 cores - 244 GB RAM - 4 GPU',
 'p316xlarge': 'V100-16XLarge - 64 cores - 488 GB RAM - 8 GPU'}

In [2]:
from dask.distributed import Client, wait
from dask_saturn import SaturnCluster
import time

n_workers = 5
cluster = SaturnCluster(n_workers=n_workers, 
                        scheduler_size='2xlarge', 
                        worker_size='4xlarge', 
                        nthreads=16)
client = Client(cluster)
cluster

[2020-10-02 16:07:23] INFO - dask-saturn | Starting cluster. Status: pending
[2020-10-02 16:07:32] INFO - dask-saturn | Starting cluster. Status: pending
[2020-10-02 16:07:43] INFO - dask-saturn | Starting cluster. Status: pending
[2020-10-02 16:08:18] INFO - dask-saturn | Cluster is ready


In [3]:
while len(client.scheduler_info()['workers']) < n_workers:
    print('Waiting for workers, got', len(client.scheduler_info()['workers']))
    time.sleep(30)
print('Done!')

Waiting for workers, got 0
Done!


In [4]:
import dask.dataframe as dd
import json
import numpy as np
import pandas as pd
import s3fs
import spacy
import scispacy

from dask.distributed import Client, progress, get_worker

from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker

## Processing

In [5]:
MODEL_KB = "umls"
# MODEL_KB = "mesh"
# MODEL_KB = "go"
# MODEL_KB = "hpo"
# MODEL_KB = "rxnorm"

In [6]:
BUCKET_NAME = "saturn-elsevierinc"

SENTENCE_FOLDER = "/".join(["s3:/", BUCKET_NAME, "incremental", "add-sents"])
ENTITIES_FOLDER = "/".join(["s3:/", BUCKET_NAME, "incremental",
                            "add-ents-{:s}".format(MODEL_KB)])

In [8]:
sentences_df = dd.read_parquet(SENTENCE_FOLDER, engine="pyarrow")
sentences_df.head()

,cord_uid,pid,sid,stext
38548,l2m8y422,T,0,Correction: Selective laser trabeculoplasty: p...
38563,kwby80nj,T,0,Publishing in the transfusion field: “
38563,kwby80nj,T,1,Like a Bridge Over Trouble Water” in a “The time
38563,kwby80nj,T,2,They Are A Changing” period
38565,9vbwzi8v,T,0,Nachfrage nicht zu bremsen


In [9]:
len(sentences_df)

228953

In [10]:
def build_nlp_pipeline(model_kb):
    nlp = spacy.load("en_core_sci_md")
    
    abbr_detector = AbbreviationDetector(nlp)
    nlp.add_pipe(abbr_detector)
    
    linker = EntityLinker(resolve_abbreviations=True,
                          filter_for_definitions=False,
                          name=model_kb)
    nlp.add_pipe(linker)
    
    return nlp


def nlp_workers():
    import traceback
    try:
        worker = get_worker()
        nlp = build_nlp_pipeline(MODEL_KB)
        worker.nlp = nlp
    except:
        return traceback.format_exc()
    return 0


def check_nlp_workers():
    worker = get_worker()
    return str(worker.nlp)


%time client.run(nlp_workers)

CPU times: user 105 ms, sys: 13 ms, total: 118 ms
Wall time: 2min 38s


{'tcp://10.0.10.240:39541': 0,
 'tcp://10.0.10.93:40085': 0,
 'tcp://10.0.11.239:37113': 0,
 'tcp://10.0.13.211:43661': 0,
 'tcp://10.0.17.21:40009': 0,
 'tcp://10.0.2.184:36513': 0,
 'tcp://10.0.25.120:38835': 0,
 'tcp://10.0.4.115:36717': 0,
 'tcp://10.0.5.12:35177': 0,
 'tcp://10.0.6.206:38385': 0}

In [11]:
client.run(check_nlp_workers)

{'tcp://10.0.10.240:39541': '<spacy.lang.en.English object at 0x7f60e267bf90>',
 'tcp://10.0.10.93:40085': '<spacy.lang.en.English object at 0x7f719c0b5c10>',
 'tcp://10.0.11.239:37113': '<spacy.lang.en.English object at 0x7fb4dc511ad0>',
 'tcp://10.0.13.211:43661': '<spacy.lang.en.English object at 0x7fdb8c23c690>',
 'tcp://10.0.17.21:40009': '<spacy.lang.en.English object at 0x7f0c0f67f210>',
 'tcp://10.0.2.184:36513': '<spacy.lang.en.English object at 0x7f16b8710610>',
 'tcp://10.0.25.120:38835': '<spacy.lang.en.English object at 0x7f85f24eafd0>',
 'tcp://10.0.4.115:36717': '<spacy.lang.en.English object at 0x7fe340092dd0>',
 'tcp://10.0.5.12:35177': '<spacy.lang.en.English object at 0x7fa48e3bbe10>',
 'tcp://10.0.6.206:38385': '<spacy.lang.en.English object at 0x7efc2c709f10>'}

In [12]:
def handle_batch(sents, nlp, model_kb):
    docs = nlp.pipe(sents, n_threads=16, batch_size=len(sents))
    ents_list = []
    try:
        for doc in docs:
            ents = []
            for eid, ent in enumerate(doc.ents):
                try:
                    kb_ents = ent._.kb_ents
                    for cid, score in kb_ents:
                        ents.append((eid, model_kb, ent.text, cid, 
                                     score, ent.start_char, ent.end_char))
                except KeyError:
                    continue
            ents_list.append(ents)
    except KeyError:
        pass
    return ents_list


def handle_partition(part):
    worker = get_worker()
    nlp = worker.nlp
    batch_size = 32
    sent_batch, ent_batch, entities = [], [], []
    for _, row in part.iterrows():
        if len(sent_batch) % batch_size == 0 and len(sent_batch) > 0:
            ent_batch = handle_batch(sent_batch, nlp, MODEL_KB)
            entities.extend(ent_batch)
            sent_batch = []
        try:
            sent_batch.append(row.stext)
        except ValueError:
            continue
    if len(sent_batch) > 0:
        ent_batch = handle_batch(sent_batch, nlp, MODEL_KB)
        entities.extend(ent_batch)
    return entities

In [13]:
entities_df = sentences_df.copy()

In [14]:
entities_df["entities"] = entities_df.map_partitions(
    lambda part: handle_partition(part), meta=("object"))
entities_df = entities_df.drop(columns=["stext"])
entities_df = entities_df.explode("entities")
entities_df = entities_df.dropna()

entities_df["eid"] = entities_df.apply(
    lambda row: row.entities[0], meta=("int"), axis=1)
entities_df["eclass"] = entities_df.apply(
    lambda row: row.entities[1], meta=("str"), axis=1)
entities_df["etext"] = entities_df.apply(
    lambda row: row.entities[2], meta=("str"), axis=1)
entities_df["elabel"] = entities_df.apply(
    lambda row: row.entities[3], meta=("str"), axis=1)
entities_df["escore"] = entities_df.apply(
    lambda row: row.entities[4], meta=("float"), axis=1)
entities_df["ent_start_char"] = entities_df.apply(
    lambda row: row.entities[5], meta=("int"), axis=1)
entities_df["ent_end_char"] = entities_df.apply(
    lambda row: row.entities[6], meta=("int"), axis=1)

entities_df = entities_df.drop(columns=["entities"])

In [15]:
entities_df.cord_uid = entities_df.cord_uid.astype(str)
entities_df.pid = entities_df.pid.astype(str)
entities_df.sid = entities_df.sid.astype(np.int32)
entities_df.eid = entities_df.eid.astype(np.int32)
entities_df.eclass = entities_df.eclass.astype(str)
entities_df.etext = entities_df.etext.astype(str)
entities_df.elabel = entities_df.elabel.astype(str)
entities_df.escore = entities_df.escore.astype(np.float32)
entities_df.ent_start_char = entities_df.ent_start_char.astype(np.int32)
entities_df.ent_end_char = entities_df.ent_end_char.astype(np.int32)

In [16]:
fs = s3fs.S3FileSystem()
if fs.exists(ENTITIES_FOLDER):
    fs.rm(ENTITIES_FOLDER, recursive=True)

In [17]:
%%time
entities_df.to_parquet(ENTITIES_FOLDER, engine="pyarrow", compression="snappy")

CPU times: user 505 ms, sys: 78.7 ms, total: 584 ms
Wall time: 11min 59s


## Verify Result

In [18]:
ENTITIES_FOLDER

's3://saturn-elsevierinc/incremental/add-ents-umls'

In [19]:
fs.du(ENTITIES_FOLDER) / 1e6

57.783464

In [21]:
entities_df = dd.read_parquet(ENTITIES_FOLDER, engine="pyarrow")
entities_df.head()

,cord_uid,pid,sid,eid,eclass,etext,elabel,escore,ent_start_char,ent_end_char
38548,l2m8y422,T,0,0,umls,Correction,C1947976,1.000000,0,10
38548,l2m8y422,T,0,0,umls,Correction,C1546923,1.000000,0,10
38548,l2m8y422,T,0,0,umls,Correction,C1705565,1.000000,0,10
38548,l2m8y422,T,0,0,umls,Correction,C0947294,0.833160,0,10
38548,l2m8y422,T,0,0,umls,Correction,C0719517,0.832146,0,10


In [22]:
len(entities_df)

6861686

In [23]:
# do this if youre done using the cluster
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
